In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from ipywidgets import interact, fixed
import ipywidgets as widgets

In [2]:
def distance(stress_x, stress_y, crack_x, crack_y):
    return ((stress_x-crack_x)**2+(stress_y-crack_y)**2)**1/2

def stress(v, r):
    return (1-1/(1+np.exp(r-v)))*(r-v)

def probability(stress_x, stress_y, radius, cracks):
    sum = []
    for name, par in cracks.items():
        crack_center_x = (par['crack_start_x'] + par['crack_end_x'])/2
        crack_center_y = (par['crack_start_y'] + par['crack_end_y'])/2
        v = distance(stress_x, stress_y, crack_center_x, crack_center_y)
        s = stress(v, radius)
        e = -(s/par['sf'])**par['m']
        sum.append(e)

        print(f'v({name}) = {v:.3f}')
        print(f's({name}) = {s:.3f}')
        print(f'e({name}) = {e:.3f}')
        print('----')
    print(f'exp_sum = {np.sum(sum):.3f}')
    print('----')

    p = 1-np.exp(np.sum(sum))
    print(f'p = {p:.3f}')
    return p

In [3]:
def stress_gradient(stress_x, stress_y, outer_radius=1.2):
    inner_radius = 0.
    # outer_radius = 1.2
    halo_color = 'red'
    center_color = 'black'  ## redish with 25% alpha
    xmin = stress_x - outer_radius
    xmax = stress_x + outer_radius
    ymin = stress_y - outer_radius
    ymax = stress_y + outer_radius
    x, y = np.meshgrid(np.linspace(xmin, xmax, 500), np.linspace(ymin, ymax, 500))
    r = np.sqrt((x - stress_x) ** 2 + (y - stress_y) ** 2)
    z = np.where(r < inner_radius, np.nan, np.clip(outer_radius - r, 0, np.inf))
    cmap = LinearSegmentedColormap.from_list('', ['#FFFFFF00', halo_color])
    cmap.set_bad(center_color)

    extend=[xmin, xmax, ymin, ymax]

    return z, extend, cmap

def find_marker_coor(base, cumulative, value):
    cumulative = np.asarray(cumulative)
    idx = (np.abs(cumulative - value)).argmin()
    return base[idx], value

In [4]:
# some fake data for distribution
data = np.random.randn(1000)
def cum_prob(data):
    # evaluate the histogram
    values, base = np.histogram(data, bins=40)
    #evaluate the cumulative
    cumulative = np.cumsum(values)/1000

    return base[:-1], cumulative


In [26]:
def plot_prob(stress_x, stress_y, stress_radius, plate_radius, cracks, i=0):
    fig, ax = plt.subplots(2,1, figsize=(5,10))

    circle1 = plt.Circle((0, 0), plate_radius, color='k', fill=False, lw=3)
    for name, par in cracks.items():
        ax[0].plot([par['crack_start_x'], par['crack_end_x']], [par['crack_start_y'], par['crack_end_y']], 'k', lw=3)

    ax[0].set_aspect(1)
    ax[0].add_artist(circle1)
    ax[0].set_xlim(-1.2*plate_radius, 1.2*plate_radius)
    ax[0].set_ylim(-1.2*plate_radius, 1.2*plate_radius)
    ax[0].set_axis_off()

    z, extend, cmap = stress_gradient(stress_x, stress_y, stress_radius)
    ax[0].imshow(z, cmap=cmap, extent=extend, origin='lower', zorder=3)

    # plot the cumulative function
    base, cumulative = cum_prob(data)
    ax[1].plot(base, cumulative, c='blue')

    # calculate the probability
    p = probability(stress_x, stress_y, stress_radius, cracks)
    marker_x, marker_y = find_marker_coor(base, cumulative, p)
    ax[1].plot(marker_x, marker_y, 'ro', markersize=10)

    plt.plot()

    # fig.savefig(f'plots/stress_gradient_{i:02d}.png', dpi=300, bbox_inches='tight')


In [27]:
# stress_x_arr = np.linspace(0,1,11)
# stress_y_arr = np.linspace(0,1,11)

# for i in range(len(stress_x_arr)):
#     stress_x = stress_x_arr[i]
#     stress_y = stress_y_arr[i]
#     plot_prob(stress_x, stress_y, i)

In [28]:
stress_x = 1.
stress_y = 1.
stress_radius = 1
plate_radius = 1

cracks = {
    'crack_1': {
            'crack_start_x': 0.5,
            'crack_start_y': 0.5,
            'crack_end_x': 0.4,
            'crack_end_y': 0.4,
            'sf': 0.3,
            'm': 3,
            },
    'crack_2': {
            'crack_start_x': -0.4,
            'crack_start_y': -0.4,
            'crack_end_x': -0.7,
            'crack_end_y': 0.4,
            'sf': 0.4,
            'm': 2,
            },
    'crack_3': {
        'crack_start_x': 0.5,
        'crack_start_y': -0.5,
        'crack_end_x': 0.2,
        'crack_end_y': -0.4,
        'sf': 0.3,
        'm': 2,
        },
    }

# plot_prob(stress_x, stress_y, stress_radius, plate_radius, cracks)

In [30]:
interact(plot_prob,
         stress_x = widgets.FloatSlider(value=0, min=-1.2*plate_radius, max=1.2*plate_radius, step=0.1),
         stress_y = widgets.FloatSlider(value=0, min=-1.2*plate_radius, max=1.2*plate_radius, step=0.1),
         stress_radius = widgets.FloatSlider(value=plate_radius, min=0.1*plate_radius, max=2*plate_radius, step=0.1),
         plate_radius = fixed(plate_radius),
         cracks = fixed(cracks),
         i = fixed(0)
         )

interactive(children=(FloatSlider(value=0.0, description='stress_x', max=1.2, min=-1.2), FloatSlider(value=0.0…

<function __main__.plot_prob(stress_x, stress_y, stress_radius, plate_radius, cracks, i=0)>